In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import json
import math
from tqdm import notebook
import time
import csv

In [ ]:
free_list = pd.read_excel('free_appid.xlsx')

In [ ]:
free_list

In [ ]:
col_list = [
'short_description',
'price',
'categories',
'genres',
'recommendations',
'release_date',
'developers',
'metacritic',
'image',
'about_the_game',
'screenshots'
]

In [ ]:
for col in col_list : 
    free_list[f'{col}'] = None

In [ ]:
# appid를 인덱스로 설정 (편의상)
free_list = free_list.set_index('appid')

In [ ]:
# 데이터가 없는 오류를 가진 appid 
no_data_appid = []

for appid in notebook.tqdm(free_list.index) :
    time.sleep(1.5)

    try:
        response = requests.get(f'http://store.steampowered.com/api/appdetails?appids={appid}&l=koreana').json()
    except:
        continue
        
    game = response[f'{appid}']
    
    # 1. success == False인 경우 제외
    if(game['success'] == False): 
        continue

    data = game['data']

    # 3. 유료 게임인 경우 제외
    if(data['is_free'] == False):
        continue
        
    # 4. DLC인 경우 제외
    if(data['type'] != 'game'):
        continue

    free_list.loc[appid, 'price'] = None


    try:
        genres = data['genres'] # 변형
        genre_list = []

        for g in genres :
            genre_list.append(g['description'])

        free_list.loc[appid, 'genres'] = genre_list
    except:
        no_data_appid.append(appid)
        continue

    try:
        categories = data['categories'] # 변형
        category_list = []
        for c in categories :
            category_list.append(c['description'])


        free_list.loc[appid, 'categories'] = category_list

    except:
        no_data_appid.append(appid)
        continue

    try:
        descriptions = data['short_description']
        free_list.loc[appid, 'short_description'] = descriptions
    except:
        no_data_appid.append(appid)

    try:
        recommendations = data['recommendations']['total']
        free_list.loc[appid, 'recommendations'] = recommendations
    except:
        no_data_appid.append(appid)

    try:
        metacritic = data['metacritic']['score']
        free_list.loc[appid, 'metacritic'] = metacritic
    except:
        no_data_appid.append(appid)


    try:
        developers = data['developers']
        free_list.loc[appid, 'developers'] = developers
    except:
        no_data_appid.append(appid)


    try:
        release = data['release_date']['date']
        free_list.loc[appid, 'release_date'] = release
    except:
        no_data_appid.append(appid)

    try:
        image = data['header_image']
        free_list.loc[appid, 'image'] = image
    except:
        no_data_appid.append(appid)
        
    try:
        about_the_game = data['about_the_game']
        free_list.loc[appid, 'about_the_game'] = about_the_game
    except:
        no_data_appid.append(appid)
        
    try:
        screenshots = data['screenshots']
        screenshot_list = []
        for s in screenshots :
            screenshot_list.append(s['path_full'])


        free_list.loc[appid, 'screenshots'] = screenshot_list
    except:
        no_data_appid.append(appid)
        



In [ ]:
# 이름 수정하기 !! 
free_list.to_excel('free_data.xlsx')

# free데이터 기존 data_eng에 병합하기

In [ ]:

# 원하는 엑셀 파일을 pd.read_excel('') 함수를 이용해 데이터프레임으로 만들어준다.

df1 = pd.read_excel('gamedata_eng.xlsx')
df2 = pd.read_excel('free_data.xlsx')

# merge 함수를 이용해 합쳐준다. left는 df1 기준으로 df2가 붙는다는 뜻.
df3 = pd.merge(df1, df2)

# 해당 데이터프레임을 엑셀로 저장해준다.
df3.to_excel('game_data_final_eng.xlsx', index=False)